# Conduit Mesh Blueprint Examples
Simulation mesh data is passed to Ascent using a shared set of conventions called the
[Mesh Blueprint] (https://llnl-conduit.readthedocs.io/en/latest/blueprint_mesh.html).

Simply described, these conventions outline a structure to follow to create Conduit trees 
that can represent a wide range of simulation mesh types (uniform grids, unstructured meshes, etc). Conduit's dynamic tree and zero-copy support make it easy to adapt existing data to conform to the Mesh Blueprint for use in tools like Ascent.

These examples outline how to create Conduit Nodes that describe simple single domain meshes and review
some of Conduits built-in mesh examples. More Mesh Blueprint examples are also detailed in Conduit's [Mesh Blueprint Examples Docs](https://llnl-conduit.readthedocs.io/en/latest/blueprint_mesh.html#examples).

In [ ]:
# remove this
import sys
sys.path.append("/Users/harrison37/Work/alpine/ascent/install-debug/python-modules/")
#sys.path.append("/Users/larsen30/research/test_builds/release/ascent/install/python-modules")

In [ ]:
# cleanup any old results
#!./cleanup.sh
# conduit + ascent imports
import conduit
import conduit.blueprint
import ascent

import math
import numpy as np

# Jupyter imports
from IPython.display import Image
# helpers we use when displaying results in the notebook
from ascent_tutorial_jupyter_utils import img_display_width
from ascent_tutorial_jupyter_utils import ImageSeqViewer

## Mesh Blueprint Example 1
### Creating a uniform grid with a single field

In [ ]:
#
# Create a 3D mesh defined on a uniform grid of points
# with a single vertex associated field named `alternating`
#

mesh = conduit.Node()

# create the coordinate set
num_per_dim = 9
mesh["coordsets/coords/type"] = "uniform";
mesh["coordsets/coords/dims/i"] = num_per_dim
mesh["coordsets/coords/dims/j"] = num_per_dim
mesh["coordsets/coords/dims/k"] = num_per_dim

# add origin and spacing to the coordset (optional)
mesh["coordsets/coords/origin/x"] = -10.0
mesh["coordsets/coords/origin/y"] = -10.0
mesh["coordsets/coords/origin/z"] = -10.0

distance_per_step = 20.0/(num_per_dim-1)

mesh["coordsets/coords/spacing/dx"] = distance_per_step
mesh["coordsets/coords/spacing/dy"] = distance_per_step
mesh["coordsets/coords/spacing/dz"] = distance_per_step

# add the topology
# this case is simple b/c it's implicitly derived from the coordinate set
mesh["topologies/topo/type"] = "uniform";
# reference the coordinate set by name
mesh["topologies/topo/coordset"] = "coords";

# create a vertex associated field named alternating
num_vertices = num_per_dim * num_per_dim * num_per_dim
vals = np.zeros(num_vertices,dtype=np.float32)
for i in range(num_vertices):
    if i%2:
        vals[i] = 0.0
    else:
        vals[i] = 1.0
mesh["fields/alternating/association"] = "vertex";
mesh["fields/alternating/topology"] = "topo";
mesh["fields/alternating/values"].set_external(vals)

In [ ]:
# print the mesh we created
print(mesh.to_yaml())

In [ ]:
# make sure the mesh we created conforms to the blueprint
verify_info = conduit.Node()
if not conduit.blueprint.mesh.verify(mesh,verify_info):
    print("Mesh Verify failed!")
    print(verify_info.to_yaml())
else:
    print("Mesh verify success!")

In [ ]:
# now lets look at the mesh with Ascent
a = ascent.Ascent()
a.open()

# publish mesh to ascent
a.publish(mesh)

# setup actions
actions = conduit.Node()
add_act = actions.append();
add_act["action"] = "add_scenes";

# declare a scene (s1) with one plot (p1) 
# to render the dataset
scenes = add_act["scenes"]
scenes["s1/plots/p1/type"] = "pseudocolor"
scenes["s1/plots/p1/field"] = "alternating"
# Set the output file name (ascent will add ".png")
scenes["s1/image_name"] = "out_ascent_render_uniform"

# print our full actions tree
print(actions.to_yaml())

# execute the actions
a.execute(actions)

# close ascent
a.close()

In [ ]:
# show the resulting image
Image("out_ascent_render_uniform.png",width=img_display_width())

## Mesh Blueprint Example 2
### Creating an unstructured tet mesh with fields

In [ ]:
#
# Create a 3D mesh defined on an explicit set of points,
# composed of two tets, with two element associated fields
#  (`var1` and `var2`)
#

mesh = conduit.Node()

# create an explicit coordinate set
x = np.array( [-1.0, 0.0, 0.0, 0.0, 1.0 ], dtype=np.float64 )
y = np.array( [0.0, -1.0, 0.0, 1.0, 0.0 ], dtype=np.float64 )
z = np.array( [ 0.0, 0.0, 1.0, 0.0, 0.0 ], dtype=np.float64 )

mesh["coordsets/coords/type"] = "explicit";
mesh["coordsets/coords/values/x"].set_external(x)
mesh["coordsets/coords/values/y"].set_external(y)
mesh["coordsets/coords/values/z"].set_external(z)

# add an unstructured topology
mesh["topologies/mesh/type"] = "unstructured"
# reference the coordinate set by name
mesh["topologies/mesh/coordset"] = "coords"
# set topology shape type
mesh["topologies/mesh/elements/shape"] = "tet"
# add a connectivity array for the tets
connectivity = np.array([0, 1, 3, 2, 4, 3, 1, 2 ],dtype=np.int64)
mesh["topologies/mesh/elements/connectivity"].set_external(connectivity)
    
var1 = np.array([0,1],dtype=np.float32)
var2 = np.array([1,0],dtype=np.float32)

# create a field named var1
mesh["fields/var1/association"] = "element"
mesh["fields/var1/topology"] = "mesh"
mesh["fields/var1/values"].set_external(var1)

# create a field named var2
mesh["fields/var2/association"] = "element"
mesh["fields/var2/topology"] = "mesh"
mesh["fields/var2/values"].set_external(var2)

In [ ]:
# print the mesh we created
print(mesh.to_yaml())

In [ ]:
# make sure the mesh we created conforms to the blueprint
verify_info = conduit.Node()
if not conduit.blueprint.mesh.verify(mesh,verify_info):
    print("Mesh Verify failed!")
    print(verify_info.to_yaml())
else:
    print("Mesh verify success!")

In [ ]:
# now lets look at the mesh with Ascent
a = ascent.Ascent()
a.open()

# publish mesh to ascent
a.publish(mesh)

# setup actions
actions = conduit.Node()
add_act = actions.append();
add_act["action"] = "add_scenes"

# declare a scene (s1) with one plot (p1) 
# to render the dataset
scenes = add_act["scenes"]
scenes["s1/plots/p1/type"] = "pseudocolor"
scenes["s1/plots/p1/field"] = "var1"
# Set the output file name (ascent will add ".png")
scenes["s1/image_name"] = "out_ascent_render_tets"

# print our full actions tree
print(actions.to_yaml())

# execute the actions
a.execute(actions)

# close ascent
a.close()

In [ ]:
# show the resulting image
Image("out_ascent_render_tets.png",width=img_display_width())

## Mesh Blueprint Example 3
### Experimenting with the built-in braid example
Related docs: [Braid Example Docs](https://llnl-conduit.readthedocs.io/en/latest/blueprint_mesh.html#braid)

In [ ]:
# The conduit blueprint library provides several 
# simple builtin examples that cover the range of
# supported coordinate sets, topologies, field etc
# 
# Here we create a mesh using the braid example
# (https://llnl-conduit.readthedocs.io/en/latest/blueprint_mesh.html#braid)
# and modify one of its fields to create a time-varying
# example

In [ ]:
# Define a function that will calcualte a time varying field
def braid_time_varying(npts_x, npts_y, npts_z, interp, res):
    if npts_z < 1:
        npts_z = 1

    npts = npts_x * npts_y * npts_z
    
    res["association"] = "vertex"
    res["topology"] = "mesh"
    vals = res["values"]
    
    dx_seed_start = 0.0
    dx_seed_end   = 5.0
    dx_seed = interp * (dx_seed_end - dx_seed_start) + dx_seed_start
    
    dy_seed_start = 0.0
    dy_seed_end   = 2.0
    dy_seed = interp * (dy_seed_end - dy_seed_start) + dy_seed_start
    
    dz_seed = 3.0

    dx = (float) (dx_seed * math.pi) / float(npts_x - 1)
    dy = (float) (dy_seed * math.pi) / float(npts_y-1)
    dz = (float) (dz_seed * math.pi) / float(npts_z-1)
    
    idx = 0
    for k in range(npts_z):
        cz =  (k * dz) - (1.5 * math.pi)
        for j in range(npts_y):
            cy =  (j * dy) - (math.pi)
            for i in range(npts_x):
                cx =  (i * dx) + (2.0 * math.pi)
                cv =  math.sin( cx ) + \
                      math.sin( cy ) +  \
                      2.0 * math.cos(math.sqrt( (cx*cx)/2.0 +cy*cy) / .75) + \
                      4.0 * math.cos( cx*cy / 4.0)
                                  
                if npts_z > 1:
                    cv += math.sin( cz ) + \
                          1.5 * math.cos(math.sqrt(cx*cx + cy*cy + cz*cz) / .75)
                vals[idx] = cv
                idx+=1

In [ ]:
# create conduit node with an example mesh using conduit blueprint's braid function
# ref: https://llnl-conduit.readthedocs.io/en/latest/blueprint_mesh.html#braid
mesh = conduit.Node()
conduit.blueprint.mesh.examples.braid("hexs",
                                      50,
                                      50,
                                      50,
                                      mesh)

In [ ]:
a = ascent.Ascent()
# open ascent
a.open()

# create our actions
actions = conduit.Node()
add_act =actions.append()
add_act["action"] = "add_scenes"

# declare a scene (s1) and plot (p1)
# to render braid field 
scenes = add_act["scenes"] 
scenes["s1/plots/p1/type"] = "pseudocolor"
scenes["s1/plots/p1/field"] = "braid"

# print our actions tree
print(actions.to_yaml())

# loop over a set of steps and 
# render a time varying version of the braid field

nsteps = 20
interps = np.linspace(0.0, 1.0, num=nsteps)
i = 0

for interp in interps:
    print("{}: interp = {}".format(i,interp))
    # update the braid field
    braid_time_varying(50,50,50,interp,mesh["fields/braid"])
    # update the mesh cycle
    mesh["state/cycle"] = i
    # Set the output file name (ascent will add ".png")
    scenes["s1/renders/r1/image_name"] = "out_ascent_render_braid_tv_%04d" % i
    scenes["s1/renders/r1/camera/azimuth"] = 25.0
    
    # publish mesh to ascent
    a.publish(mesh)

    # execute the actions
    a.execute(actions)
    
    i+=1

# close ascent
a.close()

In [ ]:
result_image_files = ["out_ascent_render_braid_tv_%04d.png" % i for i in range(nsteps)]

In [ ]:
ImageSeqViewer(result_image_files).show()